In [97]:
import numpy as np
from copy import deepcopy
import math
from sklearn.model_selection import train_test_split

activities_to_classify = {
        'Get/replace items from refrigerator/cabinets/drawers': 0,
        'Peel a cucumber': 1,
        'Clear cutting board': 2,
        'Slice a cucumber': 3,
        'Peel a potato': 4,
        'Slice a potato': 5,
        'Slice bread': 6,
        'Spread almond butter on a bread slice': 7,
        'Spread jelly on a bread slice': 8,
        'Open/close a jar of almond butter': 9,
        'Pour water from a pitcher into a glass': 10,
        'Clean a plate with a sponge': 11,
        'Clean a plate with a towel': 12,
        'Clean a pan with a sponge': 13,
        'Clean a pan with a towel': 14,
        'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 15,
        'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 16,
        'Stack on table: 3 each large/small plates, bowls': 17,
        'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 18,
        'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 19,
}

activities_to_verbs = {
        'Get/replace items from refrigerator/cabinets/drawers': 'Get/Put',
        'Peel a cucumber': 'Slice',
        'Clear cutting board': 'Clean',
        'Slice a cucumber': 'Slice',
        'Peel a potato': 'Slice',
        'Slice a potato': 'Slice',
        'Slice bread': 'Slice',
        'Spread almond butter on a bread slice': 'Spread',
        'Spread jelly on a bread slice': 'Spread',
        'Open/close a jar of almond butter': 'Open/Close',
        'Pour water from a pitcher into a glass': 'Pour',
        'Clean a plate with a sponge': 'Clean',
        'Clean a plate with a towel': 'Clean',
        'Clean a pan with a sponge': 'Clean',
        'Clean a pan with a towel': 'Clean',
        'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Get/Put',
        'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Stack',
        'Stack on table: 3 each large/small plates, bowls': 'Stack',
        'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Load',
        'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils': 'Load'
}

verbs = {
    'Get/Put': 0,
    'Load': 1,
    'Slice': 2,
    'Spread': 3,
    'Open/Close': 4,
    'Pour': 5,
    'Clean': 6,
    'Stack': 7,
}

def split_rows(data_frame):
    augmented_data = []

    for _, row in data_frame.iterrows():
        start_frame = row['start_frame']
        stop_frame = row['stop_frame']
        interval_size = 150 

        num_intervals = math.ceil((stop_frame - start_frame + 1) / interval_size)

        for i in range(num_intervals):
            new_start = start_frame + i * interval_size
            new_stop = min(new_start + interval_size - 1, stop_frame)  
            new_row = row.copy()
            new_row['start_frame'] = new_start
            new_row['stop_frame'] = new_stop
            new_row['start_timestamp'] = new_start/30
            new_row['stop_timestamp'] = new_stop/30
            augmented_data.append(new_row)

    augmented_dataframe = pd.DataFrame(augmented_data, columns=data_frame.columns)
    augmented_dataframe.reset_index(drop=True, inplace=True)

    return augmented_dataframe

def create_annotations_file(timestamp_file, annotations_file_test, annotations_file_train):
    import pandas as pd

    video_fps = 30

    timestamps = pd.read_pickle(timestamp_file)
    timestamps = timestamps.drop(
        ['myo_left_timestamps', 'myo_right_timestamps', 'myo_left_readings', 'myo_right_readings'], axis=1)
    timestamps = timestamps.reset_index()

    start_timestamp = timestamps['start'].iloc[0]
    timestamps['start_frame'] = ((timestamps['start'] - start_timestamp) * video_fps).astype(int)
    timestamps['stop_frame'] = ((timestamps['stop'] - start_timestamp) * video_fps).astype(int)
    
    data_frame_train = pd.read_pickle(annotations_file_train)
    data_frame_train = data_frame_train[data_frame_train['file'] == 'S04_1.pkl']
    data_frame_train = data_frame_train.drop(['labels'], axis=1)
    data_frame_train['type'] = 'train'

    data_frame_test = pd.read_pickle(annotations_file_test)
    data_frame_test = data_frame_test[data_frame_test['file'] == 'S04_1.pkl']
    data_frame_test = data_frame_test.drop(['labels'], axis=1)
    data_frame_test['type'] = 'test'
    
    join_data_frame = pd.concat([data_frame_train, data_frame_test], ignore_index=True)

    result = pd.merge(timestamps, join_data_frame, on='index', how='inner')
    
##--------------------------------------------------------------------------------------------------------------------
    result['uid'] = result['index']
    result['participant_id'] = 'S04'
    result['video_id'] = 'S04_01'
    result['verb'] = result['description_x']
    result['narration'] = result['description_x']
    result['verb_class'] = result['verb'].map(activities_to_classify)

    result = result[
        ['uid', 'participant_id', 'video_id', 'narration', 'start', 'stop', 'start_frame',
         'stop_frame', 'verb', 'verb_class', 'type']]
            
    result = result.sort_values(by='start_frame')
    

    result = split_rows(result)
    result['uid'] = range(0, len(result))
    
    reduced_labels = deepcopy(result)
    reduced_labels['verb'] = result['narration'].map(activities_to_verbs)
    reduced_labels['verb_class'] = result['narration'].map(activities_to_verbs).map(verbs).astype(int)
##--------------------------------------------------------------------------------------------------------------------

    test = result[result['type'] == 'test']
    test.drop('type', axis=1, inplace=True)
    test.to_csv("S04_test.csv", index=False)
    test.to_pickle("action_net_annotations/S04_test_complete.pkl")
    train = result[result['type'] == 'train']
    train.drop('type', axis=1, inplace=True)
    train.to_csv("S04_train.csv", index=False)
    train.to_pickle("action_net_annotations/S04_train_complete.pkl")
    
    test = reduced_labels[reduced_labels['type'] == 'test']
    test.drop('type', axis=1, inplace=True)
    test.to_csv("S04_test_reduced.csv", index=False)
    test.to_pickle("action_net_annotations/S04_test.pkl")
    train = reduced_labels[reduced_labels['type'] == 'train']
    train.drop('type', axis=1, inplace=True)
    train.to_csv("S04_train_reduced.csv", index=False)
    train.to_pickle("action_net_annotations/S04_train.pkl")

def create_annotations_from_emg(emg_data):
    calibration_val = 1.655239e+09
    
    emg_data = pd.read_pickle(emg_data)
    emg_data = pd.DataFrame.from_dict(emg_data, orient='index')
    emg_data = emg_data.reset_index()
    emg_data = emg_data[emg_data['subject'] == 'emg-data-S04_1']
    emg_data = emg_data.rename(columns={'subject': 'video_id', 'label': 'narration'})
    emg_data['participant_id'] = 'S04'
    emg_data['video_id'] = 'S04_1'
    emg_data['start_frame'] = ((emg_data['start_timestamp'] - calibration_val) * 30).astype(int)
    emg_data['stop_frame'] = ((emg_data['end_timestamp'] - calibration_val) * 30).astype(int)
    emg_data['verb'] = emg_data['narration']
    emg_data['verb_class'] = emg_data['narration'].map(activities_to_classify)
    emg_data['uid'] = range(0, len(emg_data))
    emg_data = emg_data.drop(['emg_data'], axis = 1)
    emg_data = emg_data[
        ['uid', 'participant_id', 'video_id', 'narration', 'start_timestamp', 'end_timestamp', 'start_frame',
         'stop_frame', 'verb', 'verb_class']]
    reduced_data = deepcopy(emg_data)
    reduced_data['verb_class'] = emg_data['narration'].map(activities_to_verbs).map(verbs).astype(int)
    train, test = train_test_split(emg_data, test_size=0.2, random_state=42)
    train_reduced, test_reduced = train_test_split(reduced_data, test_size=0.2, random_state=42)

    train.to_pickle('action_net_annotations_training/S04_train.pkl')
    test.to_pickle('action_net_annotations_training/S04_test.pkl')
    train_reduced.to_pickle('action_net_annotations_extraction/S04_train.pkl')
    test_reduced.to_pickle('action_net_annotations_extraction/S04_test.pkl')

    train.to_csv("S04_train_csv.csv", index=False)
    test_reduced.to_csv("S04_test_csv.csv", index=False)

In [98]:
create_annotations_from_emg('emg_data_preprocessed_train.pkl')

In [96]:
import pandas as pd

df = pd.read_pickle('emg_data_preprocessed_train.pkl')
emg_data = pd.DataFrame.from_dict(df, orient='index')
emg_data = emg_data.reset_index()
emg_data = emg_data[emg_data['subject'] == 'emg-data-S04_1']
emg_data

,index,subject,emg_data,label,start_timestamp,end_timestamp
1316,1316,emg-data-S04_1,"[[0.05997730459291928, 0.0711653354667301, 0.0...",Get/replace items from refrigerator/cabinets/d...,1.655240e+09,1.655240e+09
1317,1317,emg-data-S04_1,"[[0.05997730459291928, 0.08956066677500063, 0....",Get/replace items from refrigerator/cabinets/d...,1.655240e+09,1.655240e+09
1318,1318,emg-data-S04_1,"[[0.026506006912530022, 0.04682792609659714, 0...",Get/replace items from refrigerator/cabinets/d...,1.655240e+09,1.655240e+09
1319,1319,emg-data-S04_1,"[[0.013117487840374364, 0.033253936684209284, ...",Get/replace items from refrigerator/cabinets/d...,1.655240e+09,1.655240e+09
1320,1320,emg-data-S04_1,"[[-0.09399066473687034, -0.11852701733865956, ...",Get/replace items from refrigerator/cabinets/d...,1.655240e+09,1.655240e+09
...,...,...,...,...,...,...
1690,1690,emg-data-S04_1,"[[-0.09418684122066634, -0.08335868243571432, ...","Unload dishwasher: 3 each large/small plates, ...",1.655242e+09,1.655242e+09
1691,1691,emg-data-S04_1,"[[0.027982528916788763, 0.030915991717390456, ...","Unload dishwasher: 3 each large/small plates, ...",1.655242e+09,1.655242e+09
1692,1692,emg-data-S04_1,"[[-0.029508939383190103, -0.004122653471317572...","Unload dishwasher: 3 each large/small plates, ...",1.655242e+09,1.655242e+09
1693,1693,emg-data-S04_1,"[[0.10703329782925963, 0.11955563162522442, 0....","Unload dishwasher: 3 each large/small plates, ...",1.655242e+09,1.655242e+09


In [91]:
import pandas as pd

df = pd.read_pickle('action-net/ActionNet_test.pkl')

df

,index,file,description,labels
4,4,S02_2.pkl,Peel a cucumber,Peel
34,34,S04_1.pkl,Open/close a jar of almond butter,Open/Close
48,48,S05_2.pkl,Clean a plate with a towel,Clean
4,4,S05_2.pkl,Peel a cucumber,Peel
62,62,S07_1.pkl,"Stack on table: 3 each large/small plates, bowls",Stack
15,15,S06_2.pkl,Clear cutting board,Clear
34,34,S05_2.pkl,Open/close a jar of almond butter,Open/Close
13,13,S02_2.pkl,Peel a potato,Peel
23,23,S04_1.pkl,Clear cutting board,Clear
11,11,S07_1.pkl,Clear cutting board,Clear
